# Knowledge Graph Explorer

This notebook provides tools to explore and analyze the temporal knowledge graphs stored in Neo4j for each user.

## Features
- Visualize user knowledge graphs
- Query entities and relationships
- Analyze conversation statistics
- Compare multiple users
- Run custom Cypher queries

## Setup

In [ ]:
# Add project root to path
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path updated")

In [ ]:
# Import required modules
from src.visualizer import GraphVisualizer
from src.config import Neo4jConfig
from src.logging_config import setup_logging, get_logger

# Setup logging
setup_logging(log_level="INFO")
logger = get_logger(__name__)

print("✅ Imports successful")

## 1. Visualize Single User's Knowledge Graph

In [ ]:
# Create visualizer instance
viz = GraphVisualizer()

# Choose a user ID
user_id = "alice"  # Change this to the user you want to visualize

print(f"Visualizing graph for user: {user_id}")

In [ ]:
# Visualize all-time graph
html_file = viz.visualize_user_graph(
    user_id=user_id,
    days_back=None,  # None = all time
    open_browser=False  # Set to False in notebook, True for CLI
)

if html_file:
    print(f"\n✅ Visualization saved to: {html_file}")
    print("Open this file in a web browser to view the interactive graph")

In [ ]:
# Visualize last 7 days
html_file_7days = viz.visualize_user_graph(
    user_id=user_id,
    days_back=7,
    open_browser=False
)

print(f"Last 7 days graph: {html_file_7days}")

In [ ]:
# Visualize last 30 days
html_file_30days = viz.visualize_user_graph(
    user_id=user_id,
    days_back=30,
    open_browser=False
)

print(f"Last 30 days graph: {html_file_30days}")

## 2. Get User Statistics

In [ ]:
# Get statistics for the user
stats = viz.get_user_statistics(user_id)

print(f"\n📊 Statistics for user '{user_id}':")
print(f"  Episodes:      {stats['episode_count']}")
print(f"  Entities:      {stats['entity_count']}")
print(f"  Relationships: {stats['relationship_count']}")

import pandas as pd
stats_df = pd.DataFrame([stats])
display(stats_df)

## 3. Compare Multiple Users

In [ ]:
# List of users to compare
users_to_compare = ["alice", "bob", "charlie"]  # Modify as needed

# Get stats for each user
all_stats = []
for user in users_to_compare:
    try:
        stat = viz.get_user_statistics(user)
        all_stats.append(stat)
    except Exception as e:
        print(f"Could not get stats for {user}: {e}")

# Create comparison dataframe
import pandas as pd
comparison_df = pd.DataFrame(all_stats)
print("\n📊 User Comparison:")
display(comparison_df)

## 4. Run Custom Cypher Queries

In [ ]:
# Direct Neo4j access for advanced queries
from neo4j import GraphDatabase

# Example: Get all episodes for a user
query = """
MATCH (ep:Episode)
WHERE ep.group_id = $user_id
RETURN ep.name, ep.reference_time, ep.content
ORDER BY ep.reference_time DESC
LIMIT 10
"""

with viz.driver.session() as session:
    result = session.run(query, {"user_id": user_id})
    records = result.data()

print(f"\n📝 Latest 10 episodes for user '{user_id}':")
for i, record in enumerate(records, 1):
    print(f"\n{i}. {record['ep.reference_time']}")
    print(f"   {record['ep.content'][:100]}..." if record['ep.content'] else "   (no content)")

In [ ]:
# Example: Get top entities mentioned
query = """
MATCH (ep:Episode {group_id: $user_id})-[r:MENTIONS|RELATED_TO]-(entity)
WITH entity, COUNT(r) as mention_count
RETURN entity.name, mention_count
ORDER BY mention_count DESC
LIMIT 15
"""

with viz.driver.session() as session:
    result = session.run(query, {"user_id": user_id})
    records = result.data()

print(f"\n🏷️  Top entities mentioned for user '{user_id}':")
for i, record in enumerate(records, 1):
    print(f"{i:2d}. {record['entity.name']:30s} (mentioned {record['mention_count']} times)")

In [ ]:
# Example: Timeline of conversations
query = """
MATCH (ep:Episode)
WHERE ep.group_id = $user_id
RETURN DATE(ep.reference_time) as date, COUNT(ep) as episode_count
ORDER BY date DESC
LIMIT 30
"""

with viz.driver.session() as session:
    result = session.run(query, {"user_id": user_id})
    records = result.data()

import pandas as pd
df = pd.DataFrame(records)

print(f"\n📅 Conversation timeline for user '{user_id}':")
display(df)

# Plot if matplotlib is available
try:
    import matplotlib.pyplot as plt
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    plt.figure(figsize=(12, 4))
    plt.plot(df['date'], df['episode_count'], marker='o')
    plt.title(f'Conversation Activity for {user_id}')
    plt.xlabel('Date')
    plt.ylabel('Episodes')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("\n(Matplotlib not available for plotting)")

## 5. Cleanup

In [ ]:
# Close the visualizer connection
viz.close()
print("✅ Visualization session closed")

## Notes

- Graph visualization generates interactive HTML files in `/tmp/agent_visualizations/`
- Open the HTML files in a web browser for interactive exploration
- Use Cypher queries for advanced analysis
- User data is isolated by `group_id` parameter
- Time filtering available: `days_back=7`, `days_back=30`, or `None` for all time